In [1]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [2]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

os.environ["AZURE_OPENAI_API_KEY"] = config('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

urls = ('http://www.moneycontrol.com/news/business/', 
        'http://www.moneycontrol.com/news/business/markets/',
        'http://www.moneycontrol.com/news/business/stocks/',
        'http://www.moneycontrol.com/news/business/economy/',
        'http://www.moneycontrol.com/news/business/companies/',
        'http://www.moneycontrol.com/news/business/economy/',
        'http://www.moneycontrol.com/news/business/ipo/',)
tag = ('li',)
tag_classes = ('clearfix',)

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(tag,
            class_=tag_classes
        )
    ),
)

docs = loader.load()
doc = docs[0]
print(len(docs))

USER_AGENT environment variable not set, consider setting it to identify your requests.


7


In [4]:
print(doc.page_content)

  
  Strong orders from India help Suzlon climb global league tables Suzlonâ€™s wind turbine generator emerges as one of the most in demand models 
 
  Brokerage Radar: UBS downgrades L&T, analysts have mixed views about ITC Check out the latest brokerage calls and analyst comments on the stocks in action today. Our coverage includes ITC, Larsen & Toubro, Dixon Technologies and more. 
 
  Insolation Energy stock in focus on winning Rs 208-crore order Insolation Energy Share Price | The company board at its meeting held on October 15, had approved the issuance of upto 12,23,500 equity shares of face value of Rs 10 each. 
 
  Kansai Nerolac stock in focus on sale of land worth Rs 726 crore Kansai Nerolac Share Price | A meeting of the board of directors of the company will be held on November 6, 2024. 
 
  Diwali shopping: How to maximise reward points and get the most out of your budget Check your card providerâ€™s portal for exclusive festive-season redemptions. Opt for e-vouchers or g

In [5]:
from pydantic import BaseModel, Field
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a detailed summary of the content.")
    highlights: str = Field(description="Provide the highlights of the content.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [6]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [7]:
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            )

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

overview_tagging_function = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

/var/folders/q0/sfx4jy8s5bx23xxtplmtrj5c0000gp/T/ipykernel_63004/1165731906.py:5: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  convert_pydantic_to_openai_function(Overview)


In [9]:
result = tagging_chain.invoke({"input": docs})
for key, value in result.items():
    print(f"{key}")
    print(f"{value}")

summary
The documents cover a range of business news, including updates on stocks, IPOs, and the economy. Topics include Suzlon's wind turbine generator's demand, UBS downgrading L&T, Insolation Energy's stock focus after winning a Rs 208-crore order, and Kansai Nerolac's land sale. Other highlights include Tesla's Q3 performance, GIFT Nifty's muted start, and Ashika Global's Amit Jain's views on overvalued sectors. There are also updates on various companies' Q2 results, the launch of Sarvam 1 LLM, and the approval of HDFC Bank's 4.6% stake purchase in Freshara Agro Exports.
highlights
Suzlon's wind turbine generator is in high demand. UBS downgrades L&T. Insolation Energy wins Rs 208-crore order. Kansai Nerolac sells land worth Rs 726 crore. Tesla reports strong Q3. GIFT Nifty suggests a muted start. Ashika Global's Amit Jain avoids bets in 5 overvalued sectors. Various companies report Q2 results. Sarvam 1 LLM is launched. HDFC Bank buys 4.6% stake in Freshara Agro Exports.
keywords

In [10]:
from typing import List

class Tip(BaseModel):
    """Information about papers mentioned."""
    company_name: str = Field(description="The name or ticker symbol of the company.")
    sentiment: str = Field(description="The sentiment associated with the company in the text.")
    signal: str = Field(description="Infer BUY or SELL signal associated with the company in the text.")


class Info(BaseModel):
    """Information to extract"""
    tips: List[Tip]

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [21]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tip_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=tip_extraction_function, 
    function_call={"name":"Info"}
)
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [22]:
result = extraction_chain.invoke({"input": docs})
result
# for item in result:
#     for key, value in item.items():
#         print(f"{key}: {value}")
#     print("\n")

OutputParserException: Could not parse function call data: Expecting property name enclosed in double quotes: line 119 column 1 (char 3769)